# HW5 Extension - The Cache Generator

Mingsheng Xu

[GitHub Repo](https://github.com/kofyou/hw5)

## Recall: What does your generator do and what is it for?
Cache is used to accelerate memory access by exploiting spatial and temporal locality. The generator produces customizabla cache.

## Recall: What parameters does it take?

### Size Specifications:
- Totol Capacity in Words 
- Block Size in Words
- **Associativity**

### Replacement Policies:
- Round Robin or **LRU**

## Project Features Finished by 3/14

Compared to HW5:

### Chisel Cache Associativity
- [x] parameterize the associativity for Chisel module to support set-associative and fully-associative cache
- [x] organize tests for different associativity
    - heavily reused the test harness provided by HW5

### Cache Replacement Policy
- [x] parameterize the replacement policy for both Scala model and Chisel module to support LRU
    - only one implementation is finished: each block maintains a relative order
- [x] organize tests for different policies
    - heavily reused the test harness provided by HW5

## Features not finished by 3/14
- [ ] only the fist Chisel LRU implementation is finished; the second is not
- [ ] non-blocking cache

## Incremental Development

### HW5: The Base Line

1. Scala Model
    - a direct-mapped cache
    - a set-associative cache with round robin
    - a set of tests

2. Chisel Module
    - a direct-mapped cache
    - a set of tests

### Chisel General Cache
Follow the ideas of our Scala model, the set-associative cache contains a set of direct-mapped cache.
- benefit: the `tag`, `index`, and `offset` bits arrangement are the same

Organize a (capacity=32, blockSize=4, associativity=4) cache as follows:

|         | way 0        | way 1        | way 2        | way 3        |
| ------- | ------------ | ------------ | ------------ | ------------ |
| index 0 | ( )( )( )( ) | ( )( )( )( ) | ( )( )( )( ) | ( )( )( )( ) |
| index 1 | ( )( )( )( ) | ( )( )( )( ) | ( )( )( )( ) | ( )( )( )( ) |
| index 2 | ( )( )( )( ) | ( )( )( )( ) | ( )( )( )( ) | ( )( )( )( ) |
| index 3 | ( )( )( )( ) | ( )( )( )( ) | ( )( )( )( ) | ( )( )( )( ) |
| index 4 | ( )( )( )( ) | ( )( )( )( ) | ( )( )( )( ) | ( )( )( )( ) |
| index 5 | **( )( )( )( )** | **( )( )( )( )** | **( )( )( )( )** | **( )( )( )( )** |
| index 6 | ( )( )( )( ) | ( )( )( )( ) | ( )( )( )( ) | ( )( )( )( ) |
| index 7 | ( )( )( )( ) | ( )( )( )( ) | ( )( )( )( ) | ( )( )( )( ) |


- Refactor our direct-mapped cache into a way structure, `DMCacheWay`, use it for **storage**
    - stores `tags`, `valids`, `data`
    - outputs `tags`, `valid`, `data`, and `hit`
    
```Scala
class DMCacheWay(p: CacheParams) extends Module {
    val io = IO(new Bundle {
        ...
        val out = Valid(new Bundle {
            val rTag = Output(UInt(p.numTagBits.W))
            val rLine = CacheBlock()
            val validLine = Output(Bool())
            val hit = Output(Bool())
        })
    })
    
    val valids = RegInit(VecInit(Seq.fill(p.numSets)(false.B)))
    val data = SyncReadMem(p.numSets, CacheBlock())
    val tags = SyncReadMem(p.numSets, UInt(p.numTagBits.W))
}
```

- A new general cache class, `Gecache`, containing a set of ways, use it for **control**
    - sends read/write requests to ways, handles external memory access

```Scala
class GeCache(p: CacheParams) extends Cache(p) {
    val wayParams = p.copy(capacity=p.capacity/p.associativity, associativity = 1)
    // collect their ios
    val wayIOVec = VecInit(Seq.fill(p.associativity)(Module(new DMCacheWay(wayParams)).io))
}
```

How does `Gecache` make decisions?

- send requests to all ways

```Scala
        wayIOVec.foreach(wayio => wayio.in.valid := true.B)
        wayIOVec.foreach(wayio => wayio.in.bits.addr := io.in.bits.addr)
        wayIOVec.foreach(wayio => wayio.in.bits.write := false.B)
```

- hit

```Scala
        // collect all hit signals
        val hitVec = wayIOVec.map(_.out.bits.hit)
        // should be either one hit or no hit
        io.hit := hitVec.reduce((hit1,hit2) => hit1 || hit2)
```

- locate the way

```Scala
        // tranlate one-hot to decimal index
        val hitWayIndex = OHToUInt(hitVec)
        // now we can index this specific way
        wayIOVec(hitWayIndex)
```

### Fully-Associative Cache
Now we successfully suport
- the direct-mapped cache which contains a vec of one way
- the set-associative cache

What about the fully-associative cache?
- [x] it should be a vec of several ways, each of which only has one block
- [ ] it has no index bits

So, for both the model and the modules, add a special case to properly index the only block
```Scala
    val index  =
        // if not fully-associative
        if (p.associativity * p.blockSize != p.capacity)
            io.in.bits.addr(p.numOffsetBits + p.numIndexBits - 1, p.numOffsetBits)
        else
            0.U
```

### General Cache Tests
Most of the tests in HW5 are universl to different types of cache.
- encapsulate
- generalize trashing test
- add a random access test

```Scala
    def performGeneralTest(p : CacheParams) = {
        it should "be able to read (miss, then hit) a block"
        it should "be able to write miss then read hit a block"
        it should "load in a block"
        it should "be able to write to all words and then read all in cache"
        it should "handle thrashing (different associativities behave different)"
        it should "handle random accesses" in {
            val m = CacheModel(p)()
            test(new GeCache(p)).withAnnotations(Seq(WriteVcdAnnotation)) { dut =>
                for(round <- 0 until 2 * (1 << p.addrLen)) {
                    // ref: https://stackoverflow.com/q/39402567/15670192
                    // [0, 1 << p.addrLen)
                    val addr = Random.between(0, 1 << p.addrLen)
                    // [0, 1)
                    val read = Random.between(0, 2)
                    if (read == 1) {
                        performReadTest(dut, m, addr)
                    } else {
                        performWriteTest(dut, m, addr, addr)
                    }
                }
            }
        }
    }

    behavior of "Direct-Mapped GeneralCache"
    performGeneralTest(p = CacheParams(32, 4, 1))

    behavior of "Fully-Associative GeneralCache"
    performGeneralTest(p = CacheParams(32, 4, 8))

    behavior of "Set-Associative GeneralCache"
    performGeneralTest(p = CacheParams(32, 4, 2))
```

### Scala Model: Parameterize Replacement Policies

Use inheritance:
- need to find out an universal order of operations for all replacement policies so that interfaces fit in.

```Scala
abstract class SACacheModel(p: CacheParams, externalMem: ArrayBuffer[CacheBlockModel]) extends CacheModel(p, externalMem) {
    def lookUpReplPolicy(index: Int): Int
    def updatePolicyWhenMiss(addr: Int, wayIndex: Int): Unit
    def updatePolicyWhenHit(addr: Int, wayIndex: Int): Unit
    
    def wayToReplace(addr: Int): Int = {
        val (tag, index, offset) = findCacheAddressFields(addr)
        if (the index still has empty ways) {
            lookUpFillPolicy(index)
        } else {
            lookUpReplPolicy(index)
        }
    }
    
    def getReferenceToBlock(addr: Int): CacheBlockModel = {
        if (isHit == false) {
            // not hit: need too fill / replace a line
            val wayReplaceIndex = wayToReplace(addr)
            updatePolicyWhenMiss(addr, wayReplaceIndex)
            ways(wayReplaceIndex).getReferenceToBlock(addr)
        } else {
            // hit: need to update according to the policy
            updatePolicyWhenHit(addr, hitWayIndex)
            ways(hitWayIndex).getReferenceToBlock(addr)
        }
    }
}
```

The LRU Scala model implements those interfaces:

```Scala
class SALRUCacheModel(p: CacheParams, externalMem: ArrayBuffer[CacheBlockModel]) extends SACacheModel(p, externalMem) {
    val setWayUsage = Seq.fill(p.numSets)(ArrayBuffer.fill(p.associativity)(-1))

    // return the oldest index of block (way)
    def lookUpReplPolicy(index: Int): Int = {
        setWayUsage(index).indexOf(setWayUsage(index).max)
    }

    // uniformly update ranks
    def updateReplPolicy(index: Int, wayIndex: Int): Unit = {
        val theRank = setWayUsage(index)(wayIndex)
        for (wi <- 0 until p.associativity) {
            if (setWayUsage(index)(wi) < theRank) {
                setWayUsage(index)(wi) += 1
            }
        }
        setWayUsage(index)(wayIndex) = 0
    }

    def updatePolicyWhenMiss(addr: Int, wayIndex: Int): Unit = {
        val (tag, index, offset) = findCacheAddressFields(addr)
        updateReplPolicy(index, wayIndex)
        if (fillIndices(index) < p.associativity) {
            updateFillPolicy(index)
        }
    }

    def updatePolicyWhenHit(addr: Int, wayIndex: Int): Unit = {
        val (tag, index, offset) = findCacheAddressFields(addr)
        updateReplPolicy(index, wayIndex)
    }
}
```

### Scala Model: LRU Under the Hood

Each block has a relative order. Consider the ways under a given index:

- **When a block is filled up the first time**
    - set it to 0
    - for any former blocks, increment
    
| Before Filling Up                                                   | After Filling Up                                                    |
| ------------------------------------------------------------ | ------------------------------------------------------------ |
| block0: 4<br />block1: 3<br />block2: 2<br />block3: 1<br />block4: 0<br /><font color='red'>block5: -1</font><br />block6: -1<br />block7: -1 | block0: 5<br />block1: 4<br />block2: 3<br />block3: 2<br />block4: 1<br /><font color='red'>block5: 0</font><br />block6: -1<br />block7: -1 |

- **When a block hits: raise it**
    - set it to 0
    - for any blocks that are younger, increment
    - same when a new block replaces an old block

| Before Hit                                                   | After Hit                                                    |
| ------------------------------------------------------------ | ------------------------------------------------------------ |
| **block0: 3**<br />**block1: 0**<br />block2: 7<br />**block3: 1**<br />**block4: 4**<br /><font color='red'>block5: 5</font><br />**block6: 2**<br />block7: 6 | **block0: 4**<br />**block1: 1**<br />block2: 7<br />**block3: 2**<br />**block4: 5**<br /><font color='red'>block5: 0</font><br />**block6: 3**<br />block7: 6 |


- **Test**

```Scala
    it should "replace first half non-valid, update order, replace second half, and then evict the eldest" in {
        val p = CacheParams(128, 4, 4)
        val m = CacheModel(p, "LRU")()

        // fill up first half blocks in a set in order
        for (w <- 0 until p.associativity / 2) {
            val addr = w * p.numSets * p.blockSize
            assert(m.wayToReplace(addr) == w)
            testRead(m, addr, 0, false)
        }

        // age: 1,0,4,4 -> 1,0,4,4 -> 0,1,4,4
        for (w <- p.associativity / 2 - 1 to 0 by -1) {
            val addr = w * p.numSets * p.blockSize
            testRead(m, addr, 0, true)
            assert(m.wayToReplace(addr) == p.associativity / 2)
        }

        // fill up second half blocks in a set in order
        // age: 0,1,4,4 -> 1,2,0,4 -> 2,3,1,0
        for (w <- p.associativity / 2 until p.associativity) {
            // line in one set are not continuous babe
            val addr = w * p.numSets * p.blockSize
            assert(m.wayToReplace(addr) == w)
            testRead(m, addr, 0, false)
        }

        // replace index 1,0,2,3 in order
        val evictSeq = Seq(1,0,2,3)
        for (w <- 0 until p.associativity) {
            // add capacity, so tags are different
            val addr = w * p.numSets * p.blockSize + p.capacity
            assert(m.wayToReplace(addr) == evictSeq(w))
            testRead(m, addr, 0, false)
        }
    }
```

### Chisel Module: LRU Under the Hood

- **Same implementation as the Scala model:**
    - each block needs log2Ceil(p.associativity) bits
    - and a bunch of comparators, adds, and muxes

```Scala
class GeLRUCache(p: CacheParams) extends GeCache(p) {
    // ref: https://stackoverflow.com/questions/61052153/how-to-get-the-index-of-max-element-in-uint-vec-chisel
    
    // get eldest index
    def getReplIndex(): UInt = {
        LRURelativeOrder(index).indexWhere(age => age === (p.associativity - 1).U)
    }

    // compare with the selected way and update order
    def updatePolicy(wayIndex: UInt): Unit = {
        LRURelativeOrder(index).foreach(age => age := Mux(age < LRURelativeOrder(index)(wayIndex), age + 1.U, age))
        LRURelativeOrder(index)(wayIndex) := 0.U
    }
}
```

- 
    - test different combinations

```Scala
    def performGeneralTestParams(p: CacheParams) = {
        behavior of "roundRobin Direct-Mapped GeCache General Functionality"
        performGeneralTest(p, replPolicy = "roundRobin")

        behavior of "roundRobin Fully-Associative GeCache General Functionality"
        performGeneralTest(p.copy(associativity = p.numSets), replPolicy = "roundRobin")

        behavior of "roundRobin Set-Associative GeCache General Functionality"
        performGeneralTest(p.copy(associativity = p.numSets / 2), replPolicy = "roundRobin")

        behavior of "LRU Direct-Mapped GeCache General Functionality"
        performGeneralTest(p, replPolicy = "LRU")

        behavior of "LRU Fully-Associative GeCache General Functionality"
        performGeneralTest(p.copy(associativity = p.numSets), replPolicy = "LRU")

        behavior of "LRU Set-Associative GeCache General Functionality"
        performGeneralTest(p.copy(associativity = p.numSets / 2), replPolicy = "LRU")
    }

    behavior of "GeCache CacheParams(32, 4, x)"
    performGeneralTestParams(p = CacheParams(32, 4, 1))
```

- **Alternative: Example on 3-way (for simplicity) permutations and states**
    - each index has a state, denoting a certain permutation: log2Ceil(3!) bits for a state

| States   | Permutations |
| ---- | ------------ |
| 0    | 0 1 2        |
| 1    | 0 2 1        |
| 2    | 1 0 2       |
| 3    | 1 2 0        |
| 4    | 2 0 1        |
| 5    | 2 1 0        |

- 
    - all indexes share a single state transition lookup table: 3 * log2Ceil(3!) bits for an entry

| States   | Permutations | Access 0 | Access 1 | Access 2 |
| ---- | ------------ | -------- | -------- | -------- |
| 0    | 0 1 2        | 0 1 2: 0 | 1 0 2: 2 | 2 1 0: 5 |
| 1    | 0 2 1        |          |          |          |
| 2    | 1 0 2        |          |          |          |
| 3    | 1 2 0        |          |          |          |
| 4    | 2 0 1        |          |          |          |
| 5    | 2 1 0        | 0 2 1: 1 | 2 0 1: 4 | 2 1 0: 5 |

- 
    - all indexes share a single permutation-eldest lookup table: log2Ceil(3) bits for an entry

| States   | Permutations | Eldest Blocks |
| ---- | ------------ | ------------- |
| 0    | 0 1 2        | 2             |
| 1    | 0 2 1        | 1             |
| 2    | 1 0 2        | 2             |
| 3    | 1 2 0        | 1             |
| 4    | 2 0 1        | 0             |
| 5    | 2 1 0        | 0             |

- 
    - only states are stored in hardware; permutations are used only when populating the tables in Scala

- **The trade-offs and (potential) parameterization**
    - 32KB, 4B block => 8K blocks in total

|       | Per-Block Order                                             | State Machine                                                |
| ----- | ----------------------------------------------------------- | ------------------------------------------------------------ |
| 2-way | 8K * log2Ceil(2) = **8K bits**<br />extra arithmetic units  | for states:<br />8K / 2 \* log2Ceil(2!) = 4K bits<br />for transitions:<br />2! \* 2 \* log2Ceil(2!) = 4 bits<br />for translates:<br />2! \* log2Ceil(2) = 4 bits<br />in total:<br />**(4K + 8) bits** |
| 4-way | 8K * log2Ceil(4) = **16K bits**<br />extra arithmetic units | for states:<br />8K / 4 \* log2Ceil(4!) = 10K bits<br />for transitions:<br />4! \* 4 \* log2Ceil(4!) = 480 bits<br />for translates:<br />4! \* log2Ceil(4) = 48 bits<br />in total:<br />**(10K + 528) bits** |
| 8-way | 8K * log2Ceil(8) = **24K bits**<br />extra arithmetic units | for states:<br />8K / 8 \* log2Ceil(8!) = 16K bits<br />for transitions:<br />8! \* 8 \* log2Ceil(8!) = 5160960 bits = 5040K bits<br />for translates:<br />8! \* log2Ceil(8) = 120960 bits = 118.125K bits<br />in total:<br />**5174.125K bits** |

- 
    - factorial growth is drastic, but fortunately associativity might not be too big
    - special case: a victime cache
    - **conlcusion**: could parameterize the LRU implementation and hand-over the trade-offs to users

## Project Completeness:

Features not finished by 3/14:
- [ ] only the fist Chisel LRU implementation is finished; the second is not
- [ ] non-blocking cache(?)

## Ackownledgements:
- Thanks Professor Beamer and Amogh for lectures, HW5, and office hours
- Thanks Professor Litz for preparing me for this project

- [How to change color in markdown cells ipython/jupyter notebook?](https://stackoverflow.com/questions/19746350/how-to-change-color-in-markdown-cells-ipython-jupyter-notebook)
- [How to create a array/vec of Chisel modules](https://stackoverflow.com/questions/62809878/how-to-create-a-array-vec-of-chisel-modules)
- [How to do a vector of modules?](https://stackoverflow.com/questions/33621533/how-to-do-a-vector-of-modules)
- [How to get the Index of Max element in UInt Vec , Chisel](https://stackoverflow.com/questions/61052153/how-to-get-the-index-of-max-element-in-uint-vec-chisel)

## If I am on time...

1. Any interesting uses of inheritance or functional programming?

    This one on finding an empty block has not been covered by previous slides:
```Scala
        val invalidVec = wayIOVec.map(!_.out.bits.validLine)
        val atLeastOneInvalid = invalidVec.reduce((invalid1, invalid2) => invalid1 || invalid2)
        when (atLeastOneInvalid) {
            // https://www.chisel-lang.org/api/latest/chisel3/util/PriorityEncoder$.html
            replWayIndexWire := PriorityEncoder(invalidVec)
            ...
        }
```

2. Things you learned along the way
    - when to use Scala Seq and Chisel Vec (Thanks Professor Beamer and Jack!)
    - how to interact with a set of modules
    - possible implementations of LRU
    - practices on inheritence and tests

3. Advice for future students of the course
    - start early and use the slip days reasonably!